# Bootstrap, L=30

In [ ]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import Operator

from kraus_channels import KrausMap
from loss_functions import ProbabilityMSE, ProbabilityRValue
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver

from quantum_tools import  resample
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits, marginalize_counts
from spam import SPAM, InitialState, CorruptionMatrix
from quantum_circuits import integrable_circuit


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def load_data(filename, n, seed, L):
    with open(filename, 'rb') as f:
        data = pkl.load(f)


    data = marginalize_counts(data, 0)

    targets = counts_to_probs(data)
    targets_spam = targets[:6**n]
    targets_map = targets[6**n:]

    np.random.seed(seed)
    
    circuit_target = integrable_circuit(n+1, L)
    unitary = Operator(circuit_target).data

    inputs_spam, _ = generate_pauliInput_circuits(n)
        
    inputs_map, circuit_list_map = (
                generate_pauli_circuits(n, None, N=5000-6**n)
        )
    
    return inputs_spam, targets_spam, inputs_map, targets_map, unitary

def fit_spam(inputs, 
             targets,
             num_iter = 3000,
             verbose = False):
    d = targets.shape[1]
    spam_model = SPAM(init = InitialState(d),
                    povm = CorruptionMatrix(d),
                    )

    spam_opt = ModelSPAM(spam_model, tf.keras.optimizers.Adam(learning_rate=0.01))
        
    spam_opt.pretrain(100, verbose=False)

    spam_opt.train(inputs = inputs,
                    targets = targets,
                    num_iter = num_iter,
                    verbose = verbose,
                )
    
    return spam_model
    

def fit_model(inputs, 
              targets, 
              spam_model,
              num_iter = 3000,
              verbose=False):
    d = targets.shape[1]
    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        spam = spam_model,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue()], sample_freq=100),
                )

    model.train(inputs = inputs,
                targets = targets,
                inputs_val = [inputs],
                targets_val = [targets],
                num_iter = num_iter,
                N = 500,
                verbose=verbose
                )
    
    return model

In [3]:
path = '../data/chaos_exp_reorder/5q/integrable_L=30/'
n = 4
d = 2**n
L = 30
bootstrap_samples = 10

for i in tqdm(range(8,10)):

    spam_list = []
    model_list = []

    seed = 42 + i

    inputs_spam, targets_spam, inputs_map, targets_map, unitary = load_data(path + f'seed_{seed}.pkl', n, seed, L)

    tf.random.set_seed(seed)
    for j in tqdm(range(bootstrap_samples)):
        targets_spam_bs = resample(targets_spam, 12000)
        targets_map_bs = resample(targets_map, 12000)

        spam_model = fit_spam(inputs_spam, targets_spam_bs, verbose=False)
        spam_list.append(spam_model) 


        model = fit_model(inputs_map, 
                        targets_map_bs, 
                        spam_model,
                        num_iter = 3000, 
                        verbose=False)
        model_list.append(model)


    model_saver(spam_list, f'models/integrable_spam_L=30_bootstrap_seed{seed}.model')
    model_saver(model_list, f'models/integrable_model_L=30_bootstrap_seed{seed}.model')

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[0.9910910178904154]
[0.9910941400353563]
[0.9911762668211004]
[0.9911119933711737]
[0.9911046071863452]
[0.9911430918700592]
[0.9911644363049045]
[0.9911019837704645]
[0.9911373191131796]
[0.9911968103106564]


  0%|          | 0/10 [00:00<?, ?it/s]

[0.9879507338408829]
[0.9879567698697599]
[0.9878443747328648]
[0.9879600538888264]
[0.9880079004997107]
[0.9879588709519386]
[0.9879395706207506]
[0.9878339986057377]
[0.9878555667300293]
[0.9879189105772208]
